## This approach is failed.

In [27]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
import json
import random
import string
import contractions
import tensorflow as tf
import tflearn
import pickle
stemmer = LancasterStemmer()


In [33]:
with open('intent.json') as f:
    data = json.load(f)

In [48]:
# try:
#     with open("data.pickle", "rb") as f:
#         words, labels, training, output = pickle.load(f)
# except:
def tokenize(sentence):
    expanded_text = []
    for word in sentence.split():
        expanded_text.append(contractions.fix(word))
    return nltk.word_tokenize("".join(expanded_text))
def stem(word):
    return stemmer.stem(word.lower())

all_words = []
labels = []
doc_x = []
doc_y = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        w = tokenize(pattern)
        all_words.extend(w)
        doc_x.append(w)
        doc_y.append(intent["tag"])
    if intent['tag'] not in labels:
        labels.append(intent["tag"])
        
words = [stemmer.stem(w.lower()) for w in all_words if w != "?"]
words = sorted(list(set(all_words)))
labels = sorted(labels)

    
training = []
output = []

out_empty = [0 for _ in range(len(labels))]
for x, doc in enumerate(doc_x):
    bag = []
    wrds = [stemmer.stem(w.lower()) for w in doc]
    
    for w in all_words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)
    output_row = out_empty[:]
    output_row[labels.index(doc_y[x])] = 1
    
    training.append(bag)
    output.append(output_row)
    
    
training = np.array(training)
output = np.array(output)
with open("data.pickle", "wb") as f:
    pickle.dump((words, labels, training, output),f)

    

tf.compat.v1.reset_default_graph()

net = tflearn.input_data(shape=[None,len(training[0])])
net = tflearn.fully_connected(net,8)
# net = tflearn.relu(net)
net = tflearn.fully_connected(net,8)
# net = tflearn.relu(net)
net = tflearn.fully_connected(net,len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

# try:
#     model.load("Approach_2.tflearn")
# except:
model.fit(training,output,n_epoch=1500, batch_size=16, show_metric=True)
model.save("Approach_2.tflearn")

Training Step: 7499  | total loss: 1.56348 | time: 0.011s
| Adam | epoch: 1500 | loss: 1.56348 - acc: 0.3926 -- iter: 64/68
Training Step: 7500  | total loss: 1.59907 | time: 0.013s
| Adam | epoch: 1500 | loss: 1.59907 - acc: 0.3783 -- iter: 68/68
--
INFO:tensorflow:c:\Pranstech\Chat_bot\Approach_2.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [49]:
def bag_of_words(s, words):
        bag = [0 for _ in range(len(words))]
        
        s_words = nltk.word_tokenize(s)
        s_words = [stemmer.stem(w.lower()) for w in s_words]
        
        for se in s_words:
            for i,w in enumerate(words):
                if w == se:
                    bag[i] = 1
                # else:
                #     bag[i].appedn(0)
        return np.array(bag)

In [50]:
def get_response(text):
    res = model.predict([bag_of_words(text, all_words)])
    res_idx = np.argmax(res)
    
    tag = labels[res_idx]
    if res_idx > 0.9:
        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg["responses"]
    return random.sample(responses,3)    
    
    

In [70]:
get_response("Yes")

['Ohh sure, Would like to help with that...!! What type of inusrance do you need?',
 "That's great, we have Dental, Medial and Health insurance.",
 "That's great, We have multiple options for you like 'Medical', 'Dental', 'Vision'."]